In [1]:
import nltk
import contractions
import pandas as pd
import itertools
from autocorrect import Speller
from nltk import pos_tag

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from nltk.corpus import stopwords
from nltk.util import ngrams

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix

from collections import defaultdict
from collections import Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))

import re
from nltk.tokenize import word_tokenize
# import gensim
import string

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

from bert import tokenization
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, SpatialDropout1D, Dropout
from keras.initializers import Constant

# import torch

import warnings
warnings.simplefilter('ignore')


In [15]:
df = pd.read_csv("train.csv")
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [16]:
from nltk.corpus import wordnet
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


### PreProcessing Data

In [18]:
# !pip install -U spacy

# !python -m spacy download en_core_web_sm

# !python -m spacy download en_core_web_lg

In [19]:
import spacy
spc = spacy.load('en_core_web_sm')


In [20]:
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')

remove_punct=lambda x: "".join([char for char in x if char not in string.punctuation])
wn = nltk.WordNetLemmatizer()

tweets_data = list(df['text'])

# expand contractions
tweets_data = [[contractions.fix(word) for word in text.split()] for text in tweets_data]
tweets_data = [" ".join(tokens) for tokens in tweets_data]

# removing mentions
tweets_data = list(map(lambda x: re.sub(r'@[A-Za-z0-9]+','',x), tweets_data))

# cleaning numbers
tweets_data = list(map(lambda x: re.sub('[0-9]+', '', x), tweets_data))
tweets_data = list(map(lambda x: re.sub('((\x89[^s]+))', '' , x), tweets_data))

# removing URL's
tweets_data = list(map(lambda x: re.sub('((www.[^s]+)|(https?://[^s]+))',' ', x), tweets_data))



In [21]:
# unwraping abbreviations that are commonly used in social media 

abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "ama" : "ask me anything",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "gng" : "going",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [22]:
# converting abbrevations
def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

tweets_data = list(map(lambda x: convert_abbrev_in_text(x), tweets_data))
tweets_data

['Our Deeds are the Reason of this # earthquake May ALLAH Forgive us all',
 'Forest fire near La Ronge Sask . Canada',
 "All residents asked to 'shelter in place ' are being notified by officers . No other evacuation or shelter in place orders are expected",
 ', people receive # wildfires evacuation orders in California',
 'Just got sent this photo from Ruby # Alaska as smoke from # wildfires pours into a school',
 '# RockyFire Update = > California Hwy . closed in both directions due to Lake County fire - # CAfire # wildfires',
 '# flood # disaster Heavy rain causes flash flooding of streets in Manitou , Colorado Springs areas',
 'I am on top of the hill and I can see a fire in the woods ...',
 'There is an emergency evacuation happening now in the building across the street',
 'I am afraid that the tornado is coming to our area ...',
 'Three people died from the heat wave so far',
 'Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GOING TO DO WHA

In [23]:
# # takes a lot of time
# removing immediate repeating words and spellchecking 
tweets_data = [''.join(''.join(s)[:2] for _, s in itertools.groupby(text)) for text in tweets_data]
spell = Speller(lang='en')
tweets_data = [spell(text) for text in tweets_data]
tweets_data

['Our Deeds are the Reason of this # earthquake May ALL Forgive us all',
 'Forest fire near La Range Task . Canada',
 "All residents asked to 'shelter in place ' are being notified by officers . No other evacuation or shelter in place orders are expected",
 ', people receive # wildfires evacuation orders in California',
 'Just got sent this photo from Ruby # Alaska as smoke from # wildfires pours into a school',
 '# RockyFire Update = > California Hwy . closed in both directions due to Lake County fire - # Fire # wildfires',
 '# flood # disaster Heavy rain causes flash flooding of streets in Monitor , Colorado Springs areas',
 'I am on top of the hill and I can see a fire in the woods ..',
 'There is an emergency evacuation happening now in the building across the street',
 'I am afraid that the tornado is coming to our area ..',
 'Three people died from the heat wave so far',
 'Haha South Tampa is getting flooded has- AT A SECOND I IV In OUT AMA WHAT A I GNG To De WHAT A I GNG To De V

In [24]:
tweets_data_bkup = tweets_data

------------------------------------

#### More Processing of data

In [74]:
tweets_data = tweets_data_bkup

In [25]:
# removing punctuations
tweets_data = list(map(remove_punct, tweets_data))

In [26]:
# removing words like locations, organizations, money, etc which posses no value in our analysis

un_imp_words_list = [[ent for ent in spc(txt).ents if ent.label_=='MONEY' or ent.label_=='PERSON' or ent.label_=='ORG' or ent.label_=='GPE'or ent.label_=='TIME'or ent.label_=='Quantity' ] for txt in tweets_data]
# un_imp_words_list = list(filter(None, un_imp_words_list))

In [27]:
un_imp_words_list

[[May ALL Forgive],
 [Canada],
 [],
 [California],
 [Ruby, Alaska],
 [Lake County, Fire],
 [Monitor, Colorado Springs],
 [],
 [],
 [],
 [],
 [],
 [Florida, Tampabay],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [London],
 [],
 [],
 [OL],
 [],
 [],
 [],
 [],
 [],
 [],
 [Wholesale Markets],
 [],
 [Nigeria, Aba],
 [],
 [],
 [EP],
 [IEC Office],
 [Barbados  Bridgetown JAMAICA, ANT],
 [],
 [],
 [],
 [FC, Tina Vida],
 [SO PUMPED OR],
 [Chicago],
 [],
 [California],
 [],
 [],
 [First night],
 [Brighton],
 [],
 [ANT RU],
 [],
 [Niches ElBest Sanchez],
 [Turkey, Turkey Front, Diana],
 [NL],
 [],
 [tonight, Los Angeles],
 [California],
 [],
 [The Torch Publications],
 [Rene Blaze, Jacinto, Secret k  ],
 [Steve, California],
 [NowPlaying, Rene Blaze, Ian Buff],
 [Wholesale],
 [],
 [],
 [],
 [SantaClara],
 [],
 [Speeding Among Top Causes of Teen Accidents s],
 [Curry, Herman Rd, Stephenson],
 [],
 [],
 [],
 [],
 [N CABRILLO, IR],
 [Mile Marker, South Mooresville Iredale Vehicle Accident Congestion],


In [23]:
for i in range(0, len(tweets_data)):
    if un_imp_words_list[i]:
        if len(un_imp_words_list[i]) > 1:
            for word in un_imp_words_list[i]:
                if str(word) in tweets_data[i]:
                    tweets_data[i] = tweets_data[i].replace(str(word), '')
        else:
            if str(un_imp_words_list[i][0]) in tweets_data[i]:
                tweets_data[i] = tweets_data[i].replace(str(un_imp_words_list[i][0]), '')


In [24]:
tweets_data

['Our Deeds are the Reason of this  earthquake  us all',
 'Forest fire near La Range Task  ',
 'All residents asked to shelter in place  are being notified by officers  No other evacuation or shelter in place orders are expected',
 ' people receive  wildfires evacuation orders in ',
 'Just got sent this photo from    as smoke from  wildfires pours into a school',
 ' Rocky Update   California Hwy  closed in both directions due to  fire     wildfires',
 ' flood  disaster Heavy rain causes flash flooding of streets in    areas',
 'I am on top of the hill and I can see a fire in the woods ',
 'There is an emergency evacuation happening now in the building across the street',
 'I am afraid that the tornado is coming to our area ',
 'Three people died from the heat wave so far',
 'Haha South Tampa is getting flooded has AT A SECOND I IV In OUT AMA WHAT A I GNG To De WHAT A I GNG To De VC  flooding',
 ' raining  flooding      Tampa or days  I have lost count',
 ' Flood in Ago Myanmar  We arri

In [28]:
# tokenization
tweets_data = list(map(lambda x: word_tokenize(x.lower()), tweets_data))

# removing stopwords
stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')
tweets_data = [[w for w in tokens if not w in stop_words] for tokens in tweets_data]

# stemming
# tweets_data = [[snowBallStemmer.stem(word) for word in tokens] for tokens in tweets_data]

# lemmatizing
tweets_data = [[wn.lemmatize(word, get_wordnet_pos(word)) for word in tokens] for tokens in tweets_data] 

tweets_data = [" ".join(tokens) for tokens in tweets_data]

In [29]:
tweets_data

['deed reason earthquake may forgive u',
 'forest fire near la range task canada',
 'resident ask shelter place notify officer evacuation shelter place order expect',
 'people receive wildfire evacuation order california',
 'get sent photo ruby alaska smoke wildfire pours school',
 'rockyfire update california hwy close direction due lake county fire fire wildfire',
 'flood disaster heavy rain cause flash flood street monitor colorado spring area',
 'top hill see fire wood',
 'emergency evacuation happen building across street',
 'afraid tornado come area',
 'three people die heat wave far',
 'haha south tampa get flood second iv ama gng de gng de vc flood',
 'rain flood florida tampabay tampa day lose count',
 'flood ago myanmar arrive ago',
 'damage school bus multi car crash break',
 'man',
 'love fruit',
 'summer lovely',
 'car fast',
 'goal',
 'ridiculous',
 'london cool',
 'love skiing',
 'wonderful day',
 'ol',
 'way eat shit',
 'new york city last week',
 'love girlfriend',
 'c

In [82]:
# collecting repetative words
from collections import Counter
cnt = Counter()
for text in tweets_data:
    for word in text.split():
        cnt[word] += 1
FreqWords = cnt.most_common(10)
FreqWords.pop(3)
FreqWords.pop(5)
FreqWords

[('get', 429),
 ('like', 387),
 ('amp', 340),
 ('go', 312),
 ('people', 217),
 ('new', 207),
 ('one', 203),
 ('would', 185)]

#### we can see some repeating words are not useful, so we can remove them

In [30]:
df['new_text'] = tweets_data

In [86]:
# removing repetative words
FREQWORDS = set([w for (w, wc) in FreqWords])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])
df["new_text"] = df["new_text"].apply(lambda text: remove_freqwords(text))
df.head()

,id,keyword,location,text,target,new_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake u
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la range task
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident ask shelter place notify officer evac...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,receive wildfire evacuation order
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,sent photo smoke wildfire pours school


In [88]:
n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
RAREWORDS

{'bathandnortheastsomerset',
 'dei',
 'explodingkittenss',
 'explosivespacked',
 'fatherofthree',
 'gameofkittens',
 'miami',
 'overtake',
 'symptom',
 'tcotfgrayudc'}

In [89]:
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])
df["new_text"] = df["new_text"].apply(lambda text: remove_rarewords(text))
df.head()

,id,keyword,location,text,target,new_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake u
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la range task
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident ask shelter place notify officer evac...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,receive wildfire evacuation order
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,sent photo smoke wildfire pours school


#### cleaning test data

In [98]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


### Feature Extraction

In [31]:
from nltk.corpus.reader.wordnet import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline  

In [32]:
x_train, x_test, y_train, y_test = train_test_split(
    df['new_text'],
    df['target'],
    test_size=0.2,
    random_state=1)

In [33]:
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(x_train)

tf_transformer = TfidfTransformer(use_idf=False)
x_train_tf = tf_transformer.fit_transform(x_train_counts)
y_train = np.array(y_train)

# basic Naive Bayes model
clf = MultinomialNB(1.0, True, None)
clf.fit(x_train_tf, y_train) # train the classifier

# transforming test data this is the data we try our model on
x_pre_counts = count_vect.transform(x_test)
x_pre_tf = tf_transformer.transform(x_pre_counts)

predicted = clf.predict(x_pre_tf)

### All above code can be written using pipeline

In [34]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
     ('tfidf_v', TfidfVectorizer()),
     ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
    ])

In [35]:
text_clf.fit(x_train, y_train)

Pipeline(steps=[('tfidf_v', TfidfVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [36]:
predicted = text_clf.predict(x_test)

In [37]:
np.mean(predicted == y_test)

0.8056467498358503

### Logistic Regression

In [38]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(x_train_tf, y_train)

predictions = lr.predict(x_pre_tf)

In [39]:
from sklearn.metrics import classification_report

print(classification_report(predictions, y_test))

              precision    recall  f1-score   support

           0       0.89      0.79      0.83       989
           1       0.68      0.81      0.74       534

    accuracy                           0.80      1523
   macro avg       0.78      0.80      0.79      1523
weighted avg       0.81      0.80      0.80      1523



### BERT

In [40]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [45]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [42]:
# def build_model(bert_layer, max_len=512):
#     input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
#     input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
#     segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

#     _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
#     clf_output = sequence_output[:, 0, :]
    
#     if Dropout_num == 0:
#         # Without Dropout
#         out = Dense(1, activation='sigmoid')(clf_output)
#     else:
#         # With Dropout(Dropout_num), Dropout_num > 0
#         x = Dropout(Dropout_num)(clf_output)
#         out = Dense(1, activation='sigmoid')(x)

#     model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
#     model.compile(Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    
#     return model

#### Load and Preprocess
* Load BERT from the Tensorflow Hub
* Load CSV files containing training data
* Load tokenizer from the bert layer
* Encode the text into tokens, masks, and segment flags

In [7]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

Wall time: 17 s


In [8]:
# train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
# submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [11]:
# import tensorflow as tf
# tf.gfile = tf.io.gfile

In [13]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [43]:
train_input = bert_encode(df.new_text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = df.target.values

In [46]:
model_BERT = build_model(bert_layer, max_len=160)
model_BERT.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 160)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 160)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 160)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 1024),       335141889   ['input_word_ids[0][0]',         
                                 (None, 160, 1024)]               'input_mask[0][0]',         

In [ ]:
# Train BERT model
checkpoint = ModelCheckpoint('model_BERT.h5', monitor='val_loss', save_best_only=True)

train_history = model_BERT.fit(
    train_input, train_labels,
    validation_split = 0.2,
    epochs = 3, # recomended 3-5 epochs
    callbacks=[checkpoint],
    batch_size = 6
)

Epoch 1/3
